In [19]:
import pandas as pd
#import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt 
import folium 
import math
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances,haversine_distances
from typing import List
from collections import defaultdict 
from tqdm.notebook import tqdm
import networkx as nx
from math import radians
import json
from math import pi

In [20]:
#Read the dataset 

fields = ["city","lat","lng","country","population"]
df = pd.read_excel('worldcities.xlsx', usecols=fields,engine="openpyxl")
airports_df= pd.read_csv('airports.csv')
Paths_df= pd.read_csv('routes.csv')
number_city = len(df)
#sostituisco il nome della prima colonna del data frame airport_df per comodità (tolgo lo spazio)
aux_list = list(airports_df.columns)
aux_list[0] = 'Airport_ID'
airports_df.columns = aux_list
#elimino gli spazi dai nomi delle colonne di Paths_df
Paths_df.columns =[column.replace(" ", "_") for column in Paths_df.columns]
#caratterizzo i dati 
#elimino da paths_df tutte le colonne che hanno dati rilevanti mancanti
Paths_df = Paths_df[(Paths_df['Destination_airport_ID'] != '\\N') & (Paths_df['Source_airport_ID'] != '\\N')]
#converto i dati nelle colonne "Source airport ID", "Destination airport ID" in interi
Paths_df[["Source_airport_ID", "Destination_airport_ID"]] = Paths_df[["Source_airport_ID", "Destination_airport_ID"]].apply(pd.to_numeric)
#creo le colonne 'Latitude_rad' 'Longitude_rad' applicando la funzione radians(x) alle colonne "Latitude" "Longitude"
#questo per evitare in futuro di doverle calcolare più volte inutilmente
airports_df['Latitude_rad'] = airports_df["Latitude"].apply(lambda x: radians(x))
airports_df['Longitude_rad'] = airports_df["Longitude"].apply(lambda x: radians(x))
df['lat_rad'] = df['lat'].apply(lambda x: radians(x))
df['lng_rad'] = df['lng'].apply(lambda x: radians(x))

In [21]:
def int_to_list (a):
    #converte un intero in una lista con quell'intero come unico elemento
    if type (a) == int:
        return([a])
    else:
        return(a)
def List_comparison (a,b):
    #prende in input due liste, se hanno almeno un elemento in comune restituisce True, altrimenti False
    result = False
    for e in a:
        if e in b:
            result = True
    return(result)

In [22]:
#trovo il codice dell'aeroporto della corrispettiva città, se sono presenti più aeroporti per una città vengono 
#elencati in una lista
#l'associazione è fatta per le coppie (city,country) , non posso associare solamente usanto city come discriminante
#in quanto, per esempio, possono esserci due città con lo stesso nome (Valencia esiste sia in Spagna sia in Venezuela)
#è possibile che vi siano città senza alcun aereoporto associato dopo questa fase per 2 motivi:
# 1: non esiste effettivamente alcun aeroporto
# 2: i dataframe df e airports_df non hanno rappresentazioni univoce per la coppia city-country in quanto certe 
#rappresentazioni di country non sono coerenti (es: South Korea in df e Korea,South in airports_df)

lista = [[]] *(len(df.index))
dictionary_aux ={}
for e in tqdm(airports_df.index):
    try:
        dictionary_aux[(airports_df.loc[e,'City'],airports_df.loc[e,'Country'])].append(airports_df.loc[e,'Airport_ID'])
    except KeyError:
        dictionary_aux[(airports_df.loc[e,'City'],airports_df.loc[e,'Country'])] = [airports_df.loc[e,'Airport_ID']]
        
for e in tqdm(df.index):
        try:
            lista[e] = dictionary_aux[(df.loc[e,'city'],df.loc[e,'country'])]
        except KeyError:
            lista[e] = []
df['airport_id'] = lista


In [23]:
#per tutte le città senza aeroporto calcolo l'areoporto più vicino nello stesso paese
#se non esiste un aeroporto in quel paese cerco l'aeroporto più vicino
df['own_airport'] = np.ones(len(df.index)) 
aux_list = list(set(airports_df['Country']))
aux_dictionary_1 = {}
for e in tqdm(aux_list):
        aux_dictionary_1[e] = list(airports_df[airports_df['Country'] == e].index)
    
    
for e in tqdm(df.index):
    if df.loc[e,'airport_id'] ==[]:
        country = df.loc[e].country
        #aux_airports contiene tutti gli aeroporti presenti nel paese della città che sto analizzando
        try:
            aux_airports = airports_df.loc[aux_dictionary_1[country]]
            aux_airports['distance'] = [(haversine_distances([[df.loc[e,'lat_rad'],df.loc[e,'lng_rad']],
                                                         [aux_airports.loc[x,'Latitude_rad'],
                                                          aux_airports.loc[x,'Longitude_rad']]])[0][1])
                                    for x in aux_airports.index]
            
            df.loc[e,'airport_id'] = int(aux_airports[aux_airports.distance ==  aux_airports['distance'].min()].Airport_ID)
            if df.loc[e,'city'] != aux_airports[aux_airports.distance ==  aux_airports['distance'].min()].City.item():
                df.loc[e,'own_airport'] = 0
        except KeyError:
            
            #per ogni aeroporto calcola la distanza da quella città e prelevo solo gli aereoporti di quella 
            #città oppure il più vicino
            lat = df.loc[e,'lat_rad']
            lng = df.loc[e,'lng_rad']
            airports_near_found = False
            width = 0.01
            while airports_near_found == False:
                latmin = lat - width
                latmax = lat + width
                lngmin = lng - width
                lngmax = lng + width
                if lngmin < -pi:
                    lngmin = 2 *pi - lngmin
                    aux_airports = airports_df[(airports_df['Latitude_rad'] > latmin) & (airports_df['Latitude_rad'] < latmax) 
                                                   & ((airports_df['Longitude_rad'] > lngmin) 
                                                   | (airports_df['Longitude_rad'] < lngmax))]
                elif lngmax > pi:
                    lngmax = lngmax - 2 * pi
                    aux_airports = airports_df[(airports_df['Latitude_rad'] > latmin) & (airports_df['Latitude_rad'] < latmax) 
                                                   & ((airports_df['Longitude_rad'] > lngmin) 
                                                   | (airports_df['Longitude_rad'] < lngmax))]
                else:
                    aux_airports = airports_df[(airports_df['Latitude_rad'] > latmin) & (airports_df['Latitude_rad'] < latmax)
                                                   & (airports_df['Longitude_rad'] > lngmin) 
                                                   & (airports_df['Longitude_rad'] < lngmax)]
                if len(aux_airports) == 0:
                    width += 0.01
                else:
                    airports_near_found = True
                    
            aux_airports['distance'] = [haversine_distances([[lat,lng],
                                                         [aux_airports.loc[x,'Latitude_rad'],
                                                          aux_airports.loc[x,'Longitude_rad']]])[0][1]
                                    for x in aux_airports.index]
            
            #ordino gli aeroporti per distanza
            aux_airports = aux_airports.sort_values(by=['distance'], ascending = 0, inplace = False)
            #trovo il primo aeroporto che non appartiene a quella città
            count = 0
            find = 0
            #l'aeroporto più vicino è effettivamente associato alla città analizzata se ne esiste uno oppure 
            #potrebbe essere associato ad un altra città limitrofa
            while find == 0:
                try: 
                    if aux_airports.iloc[count]['City'] != df.loc[e,'city']:
                        find = 1
                except IndexError:
                    find = 1
                if find == 0:
                    count += 1
            #se count >0 ho trovato aereoporti effettivamente associati alla città
            #se count è zero ho trovato l'aereoporto più vicino che tuttavia è in una città limitrofa e devo considerare
            #che devo fare un extra tragitto epr raggiungere l'aereoporto
            if count > 0:
                aux_airports = aux_airports.iloc[:count]
                df.loc[e,'airport_id'] = [a for a in aux_airports['Airport_ID']]
            if count == 0:
                df.loc[e,'own_airport'] = 0
                df.loc[e,'airport_id'] = aux_airports.iloc[0]['Airport_ID']

df['airport_id'] = df['airport_id'].apply(lambda x: int_to_list(x))             

<ipython-input-23-c11ae4e2be03>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux_airports['distance'] = [haversine_distances([[lat,lng],


In [ ]:
#salvo il dataframe ricalcolato per evitare ulteriori inizializzazioni
df.to_csv (r'export_dataframe_1.csv', index = False, header=True)

In [ ]:
#riprendo il dataframe già calcolato
df = pd.read_csv('export_dataframe_1.csv')
df['airport_id'] = df['airport_id'].apply(lambda x: json.loads(x))
df['airport_id'] = df['airport_id'].apply(lambda x: int_to_list(x))

In [ ]:
dictionary_aux ={}
for e in tqdm(Paths_df.index):
    try:
        dictionary_path[(Paths_df.loc[e,'Source_airport_ID'],Paths_df.loc[e,'Destination_airport_ID'])]

        
        
    

In [47]:
#calcolo la distanza di ogni tragitto
dictionary_airports = {}
Paths_df['path_lenght'] = np.zeros(len(Paths_df.index))
for e in tqdm(airports_df.index):
    dictionary_airports[airports_df.loc[e,'Airport_ID']] = [airports_df.loc[e,'Latitude'],airports_df.loc[e,'Longitude']]
    
    
for e in tqdm(Paths_df.index):
    start = Paths_df.loc[e,'Source_airport_ID']
    end = Paths_df.loc[e,'Destination_airport_ID']
    Paths_df.loc[e,'path_lenght'] = haversine_distances([[dictionary_airports[start][0]
                                                         ,dictionary_airports[start][1]]
                                                        ,[dictionary_airports[end][0]
                                                         ,dictionary_airports[end][1]]])[0][1]
    #nel caso non ci sia alcun aeroporto correlato alla tratta,sia di partenzia che di arrivo, elimino quella tratta

    

In [ ]:
dictionary = {}
earth_diameter =12742 #[km]
Plane_Speed  = 900 #[km/h]
#per ogni città cerco i 30 percorsi disponibili che hanno come partenza un aereoporto di quella città ordinati per distanza
for e in tqdm(df.index):
        airports_list = df.loc[e,'airport_id']
        Paths_Available = Paths_df.query('Source_airport_ID in @airports_list').sort_values(by=['path_lenght'], 
                                                                                          ascending = 1, inplace = False).iloc[0:30]
        #ci sono dei path che presentano stessa partenza e stesso arrivo ma con modelli di aereo differenti, 
        #elimino quindi i duplicati che si possono creare per questo effetto
        #trasformando i risultati in un insieme e poi di nuovo in una lista
        Paths_Available_List = list(set([h for h in Paths_Available['Destination_airport_ID']]))
        #tra tutte le città disponibili, quelle raggiungibili sono quelle che hanno come aereoporto associato un aereoporto
        #di destinazione contenuto in Paths_Avaible_List
        City_reacheable = df
        City_reacheable['Linked'] = City_reacheable['airport_id'].apply(lambda x: List_comparison(Paths_Available_List,x))
        City_reacheable = City_reacheable[City_reacheable['Linked'] == True]
        links_list = []
        start_latutude = df.loc[e,'lat']
        start_longitude = df.loc[e,'lng']
        for t in City_reacheable.index:
            #2 ore aggiuntive se la città non ha l'aeroporto
            distance = haversine_distances([[start_latutude,start_longitude]
                                            ,[City_reacheable.loc[t,'lat']
                                              ,City_reacheable.loc[t,'lng']]])[0][1]
            time = distance * (earth_diameter/2) / Plane_Speed
            time_path = 2 * (City_reacheable.loc[t,'own_airport']+df.loc[e,'own_airport']) + time 
            links_list.append((t,time_path))
        dictionary[e] = links_list    
            

In [53]:
dictionary_airports[1]

KeyError: (1, 2)